<a href="https://colab.research.google.com/github/gondow/rust-future/blob/main/notebooks/rust_future.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 準備

Rust環境をセットアップする（1分弱ほど時間かかります）
- `export PATH=$HOME/.cargo/bin:$PATH`が効かないのでPythonでやってます
- ジェネレータを使うために nightly にしてます（nightly機能は不安定なので，以下のコードがそのままでは動かなくなる可能性大です．2025年6月には動いてました．）

In [ ]:
!curl --proto '=https' --tlsv1.2 -sSf https://sh.rustup.rs | sh -s -- -y
import os
os.environ['PATH'] = f"{os.environ['HOME']}/.cargo/bin:" + os.environ['PATH']
!rustup install nightly
!rustup default nightly
!rustc --version

info: downloading installer
warn: It looks like you have an existing rustup settings file at:
warn: /root/.rustup/settings.toml
warn: Rustup will install the default toolchain as specified in the settings file,
warn: instead of the one inferred from the default host triple.
info: profile set to 'default'
info: default host triple is x86_64-unknown-linux-gnu
warn: Updating existing toolchain, profile choice will be ignored
info: syncing channel updates for 'nightly-x86_64-unknown-linux-gnu'
info: default toolchain set to 'nightly-x86_64-unknown-linux-gnu'

  nightly-x86_64-unknown-linux-gnu unchanged - rustc 1.90.0-nightly (bdaba05a9 2025-06-27)


Rust is installed now. Great!

To get started you may need to restart your current shell.
This would reload your PATH environment variable to include
Cargo's bin directory ($HOME/.cargo/bin).

To configure your current shell, you need to source
the corresponding env file under $HOME/.cargo.

This is usually done by running one of the following

ファイル `hello.rs` を作成する

In [ ]:
%%writefile hello.rs
fn main() {
    println!("Hello, world!");
}

Writing hello.rs


`hello.rs` をコンパイルして実行する

In [ ]:
!rustc hello.rs
!./hello

Hello, world!


# Iterator

In [ ]:
%%writefile iterator.rs
fn main() {
    for i in 0..3 {
        println!("{}", i);
    }
}

Writing iterator.rs


In [ ]:
!rustc iterator.rs
!./iterator

0
1
2


`0..3`はイテレータ．イテレータは`next()`を呼ぶと「次の値」をオプショナル型として返す．

In [ ]:
%%writefile generator2.rs
fn main() {
   let mut iter = 0..3;
   println!("{:?}", iter.next()); // Some(0)
   println!("{:?}", iter.next()); // Some(1)
   println!("{:?}", iter.next()); // Some(2)
   println!("{:?}", iter.next()); // None
}

Writing generator2.rs


In [ ]:
!rustc generator2.rs
!./generator2

Some(0)
Some(1)
Some(2)
None


自分でもイテレータを定義できる．以下はCounterイテレータの定義例．途中でreturnして値を返して，次にnext()を呼ばれる時，イテレータ構造体の`count`の値が状態として保存されているので，順番にインクリメントされた値が返る．

In [ ]:
%%writefile counter.rs
struct Counter {
    count: usize,
}
impl Iterator for Counter {
    type Item = usize;
    fn next(&mut self) -> Option<usize> {
        if self.count < 3 {
            self.count += 1;
            Some(self.count)
        } else {
            None
        }
    }
}
fn main() {
    let mut counter = Counter {count: 0};
    println!("{:?}", counter.next ()); // Some(1)
    println!("{:?}", counter.next ()); // Some(2)
    println!("{:?}", counter.next ()); // Some(3)
    println!("{:?}", counter.next ()); // None
}

Overwriting counter.rs


In [ ]:
!rustc counter.rs
!./counter

Some(1)
Some(2)
Some(3)
None


Iteratorの定義は↓こうなっている．
```
pub trait Iterator {
    type Item;
    fn next(&mut self) -> Option<Self::Item>;
    // 他にもたくさん
}
```
詳しくは[公式の説明](https://doc.rust-lang.org/std/iter/trait.Iterator.html)を見てね．

# Generator (Coroutine)

In [ ]:
%%writefile generator.rs
#![feature(coroutines, coroutine_trait)]
#![feature(stmt_expr_attributes)]
use std::pin::Pin;
use std::ops::{Coroutine};
fn main() {
    let mut coro = #[coroutine] || {
        println!("Start");
        yield 1;
        println!("Middle");
        yield 2;
        println!("End");
        return 3;
    };
    let mut coro = Pin::new(&mut coro);
    println!("{:?}", coro.as_mut().resume(())); // Yielded(1)
    println!("{:?}", coro.as_mut().resume(())); // Yielded(2)
    println!("{:?}", coro.as_mut().resume(())); // Complete(3)
}

Overwriting generator.rs


In [ ]:
!rustc generator.rs
!./generator

Start
Yielded(1)
Middle
Yielded(2)
End
Complete(3)


ジェネレータ（コルーチン）とは
- 状態を保存して途中で一時停止(yield)して値を返し，後から再開(resume)できる関数（コルーチン）
- 上記の例では`yield 1;`で値1を返すと，次に`resume`された時に「`yield 1;`の直後」から実行が再開される
- `yield`すると，（どこまで実行したかの状態を保存した後で）制御はジェネレータの呼び出し側に戻る
- ジェネレータでは`yield`を使えるが，イテレータでは使えない
- 現在，ジェネレータは非推奨機能であり，`async/await`を使うことが推奨されている（なじぇ！？→自己参照でunsafeになるので`async/await`で抽象化（隠蔽）したいらしい）

ジェネレータとイテレータの主な違い：

| 観点           | イテレータ（`Iterator`）                  | ジェネレータ（`Coroutine` / 旧 `Generator`）                   |   |                                     |
| ------------ | ---------------------------------- | ----------------------------------------------------- | - | ----------------------------------- |
| 定義方法         | `struct` + `impl Iterator for ...` | \`let g =                                             \|\| { yield ...; return ... };\`（クロージャ） |
| トレイト名        | `Iterator`                         | `Coroutine`（以前は `Generator`）                          |   |                                     |
| メソッド名        | `next()`                           | `resume(arg)`                                         |   |                                     |
| 戻り値の型        | `Option<T>`                        | `CoroutineState<Y, R>` (`Yielded(Y)` / `Complete(R)`) |   |                                     |
| 状態の保存        | ユーザーが構造体で管理する                      | Rust ランタイムが自動的に状態（中断点）を保存                             |   |                                     |
| 停止と再開        | 自前で実装（ループ・分岐）                      | `yield` と `resume` による自動停止・再開                         |   |                                     |
| 双方向通信（値の送受信） | 基本不可（値を返すだけ）                       | `resume(arg)` → `yield` で双方向通信可能                      |   |                                     |
| 実装の容易さ       | 明示的なステートマシンを書く必要がある                | `yield` を書くだけで状態機械を自動生成できる                            |   |                                     |
| 安定性          | ✅ 安定版で使える                          | 🚧 Nightly 限定。`#![feature(coroutines)]` 必須            |   |                                     |
| 用途           | イテレーション（for文など）                    | 複雑な制御・状態遷移・非同期処理の構築など                                 |   |                                     |


# Future

`Future`は（都合上，まずは単純化した`SimpleFuture`を使う）
- 実行中・停止中・完了済みなどの「状態を持つ」
- 関数`poll`を持つ．内部的にジェネレータを持っていて，`yield`すると（つまり計算が完全に終了しないと），`Poll<T>`型の`Pending`を返す
- 一方，`poll`が`return`すると（つまり計算が完全に終了すると），`Poll<T>`型の`Ready(T)`を返す
- 本来は非同期な状態を扱うが，単純化したSimpleFutureは同期的な状態しか扱えない
  - 外から`poll`が呼ばれて（つまりポーリングして），同期的に計算が進む
  - `poll`の第2引数の`Context`経由で`waker`が渡されると非同期な状態（非同期的に`poll`を叩くこと）が可能になる

In [ ]:
trait SimpleFuture {
    type Output;
    fn poll(&mut self) -> Poll<Self::Output>;
}
enum Poll<T> {
    Ready(T),
    Pending,
}

In [16]:
%%writefile simplefuture.rs
#![feature(coroutines, coroutine_trait)]
#![feature(stmt_expr_attributes)]

use std::ops::{Coroutine, CoroutineState};
use std::pin::Pin;
use std::task::Poll;
use std::thread::sleep;
use std::time::Duration;
struct SimpleFuture {
    state: u8,
    pinned: Pin<Box<dyn Coroutine<Yield = u8, Return = u8>>>
}
type SimpleOutput = &'static str;
impl SimpleFuture {
    fn new() -> Self {
       let coro = #[coroutine] || {
           println!("Start");
           yield 1;
           println!("Middle");
           yield 2;
           println!("End");
           return 3;
        };
        Self { state: 0, pinned: Box::pin(coro), }
    }
    fn poll(mut self: Pin<&mut Self>) -> Poll<SimpleOutput> {
        match self.pinned.as_mut().resume(()) {
            CoroutineState::Yielded(val) => {
                println!("Yielded: {}->{}", self.state, val);
                self.state = val;
                Poll::Pending
            }
            CoroutineState::Complete(val) => {
                println!("Complete: {}->{}", self.state, val);
                self.state = val;
                Poll::Ready("Done")
            }
        }
    }
}

fn main() {
    let mut fut = SimpleFuture::new();
    let mut pinned = unsafe { Pin::new_unchecked(&mut fut) };

    loop {
        println!("loop...");
        match pinned.as_mut().poll() { // pollを呼ぶ
            Poll::Ready(val) => {
                println!("Coroutine returned: {}", val);
                break; // 計算が完了したらループを抜ける
            }
            Poll::Pending => {
                println!("Coroutine yielded");
                // 計算未完了なので，2秒待ってから次のループへ
            }
        }
        sleep(Duration::from_secs(2));
    }
}

Writing simplefuture.rs


In [17]:
!rustc simplefuture.rs
!./simplefuture

loop...
Start
Yielded: 0->1
Coroutine yielded
loop...
Middle
Yielded: 1->2
Coroutine yielded
loop...
End
Complete: 2->3
Coroutine returned: Done


# async/await

`async/await`とは
- `async`は非同期な処理（`Future`）を返す関数やブロック
- `await`は「その処理」の完了を待つ（けど，内部的には`yield`すると，`poll`を呼んだ側に制御が戻るので待ってない）

In [ ]:
async fn foo () {
    ...
}

は以下のコードと同じです．

In [ ]:
fn foo () -> impl Future<Output = ()> {
    async { ... }
}

`async`ブロックは
- `Future`を返します．`Future`の中にはジェネレータが暗黙的に入ってます．
- 返した`Future`は他の誰か（エグゼキュータ）が`poll`することで実行が進みます．

`future.await`はざっくり次の擬似コードと同じです．
- 繰り返し実行して`Pending`の時は`yield`し（実際には`poll`内部で`yield`してるはず），`Ready(val)`ならループを抜けて，`await`の次に制御が進みます．
- ループ中で`yield`した際は`poll`を呼び出した人に制御が戻ります．

In [ ]:
loop {
    match future.poll () {
        Poll::Ready(val) => { break; }
        Poll::Pending => { yield; }
    }
}

`simplefuture.rs`を`async/await`を使って書き直すと，おおよそ以下の`foo/main.rs`となります．
- `yield`するにはFutureが必要なので，ここでは`tokio::task::yield_now()`が返すFutureに`await`しています．
- 見て分かる通り，`async/await`を使うとコードがシンプルになります．

In [44]:
!rm -rf foo
!cargo new foo
!cd foo; rm -rf src

    Creating binary (application) `foo` package
note: see more `Cargo.toml` keys and their definitions at https://doc.rust-lang.org/cargo/reference/manifest.html


In [45]:
%%writefile foo/Cargo.toml
[package]
name = "foo"
version = "0.1.0"
edition = "2024"
[[bin]]
name = "main"
path = "main.rs"
[dependencies]
tokio = { version = "1", features = ["full"] }
mini-redis = "0.4"

Overwriting foo/Cargo.toml


In [46]:
%%writefile foo/main.rs
use tokio::time::Duration;
use tokio::time::sleep;
async fn simple_future() -> &'static str {
    println!("Start");
    println!("Yielded: 0 -> 1");
    tokio::task::yield_now().await;
    sleep(Duration::from_secs(2)).await;

    println!("Middle");
    println!("Yielded: 1 -> 2");
    tokio::task::yield_now().await;
    sleep(Duration::from_secs(2)).await;

    println!("End");
    tokio::task::yield_now().await;
    println!("Complete: 2 -> 3");

    "Done"
}
#[tokio::main]
async fn main() {
    let result = simple_future().await;
    println!("Coroutine returned: {}", result);
}

Writing foo/main.rs


In [47]:
!cd foo;  cargo add tokio --features full; cargo build; cargo run

ls: cannot access 'target': No such file or directory
[package]
name = "foo"
version = "0.1.0"
edition = "2024"
[[bin]]
name = "main"
path = "main.rs"
[dependencies]
tokio = { version = "1", features = ["full"] }
mini-redis = "0.4"
    Updating crates.io index
      Adding tokio v1 to dependencies
             Features as of v1.0.1:
             + bytes
             + fs
             + full
             + io-std
             + io-util
             + libc
             + macros
             + memchr
             + net
             + num_cpus
             + once_cell
             + parking_lot
             + process
             + rt
             + rt-multi-thread
             + signal
             + signal-hook-registry
             + sync
             + time
             + tokio-macros
             - mio
             - test-util
             - tracing
             - winapi
    Updating crates.io index
     Locking 117 packages to latest Rust 1.90.0-nightly compatible versions
   Compili

ここまでの話はすべて**同期的**でした．
- `poll`を呼び出す人（エグゼキュータ）がどのタイミングでどのくらいの頻度で`poll`を呼び出すかは，エグゼキュータの実装依存です．
- 頻繁に`poll`を呼ぶと（遅延は減りますが）空振りが多くなりCPUの無駄遣いになります
- 一方，頻度を下げると，次の実行準備ができてから実行されるまでの遅延時間が増えます

これを避けるには「イベントが発生したら，それに応じて「`poll`してね」とお願いしてもらうのが良いわけで，それが`wake`です．
- （エグゼキュータでもアプリでもなく）非同期イベントを監視してる人（例：タイマー，I/O）が`wake`を呼ぶ
- `wake`すると，エグゼキュータが`poll`を呼ぶ

| 処理方式            | 特徴                              |
| --------------- | ------------------------------- |
| **ポーリング**       | イベントが起きているかを何度もチェック（CPU浪費、応答遅延） |
| **非同期（イベント駆動）** | イベントが「起きたときにだけ」通知が来る（効率的、即応）    |
